<a href="https://colab.research.google.com/github/avinashpulapaka/streamlit-example/blob/master/AOS_Daily_Reporting_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install psycopg2
import pandas as pd
import datetime
#import pandasql
import numpy as np
import psycopg2
from datetime import timedelta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
is_date_range = True # true => date range, false => single day
if is_date_range:
    start_date = datetime.date(2022,5,26) # ENTER DATE HERE
    end_date = datetime.date(2022,6,2) # ENTER DATE HERE
    str_start_date = str(start_date)
    str_end_date = str(end_date)
else: 
    start_date = datetime.date(2022,3,10) # ENTER DATE HERE
    end_date = start_date + datetime.timedelta(days=1) 
    str_start_date = str(start_date)
    str_end_date = str(end_date)

### Function for submitted to publishers,editing,writing,under review,ready for review

In [19]:
def get_column(code,column_name,str_start_date,str_end_date):
    #establishing the connection
    conn = psycopg2.connect(
       database="ee-dashboard", user='mustafa', password='qNNsfVEX!^AA', host='35.245.120.183', port= '5432'
    )

    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()

    # Write query
    query = f"""
    select publishers.name as publisher, COUNT(actions_content.articleid) as {column_name}
    from actions_content
    inner join articles on actions_content.articleid = articles.id and actions_content.actionresult = {code} 
    and actions_content.createdat between '{str_start_date}' and '{str_end_date}'
    right join publishers on publishers.id = articles.publisherid
    group by publishers.name
    order by publishers.name
    """

    #Executing an MYSQL function using the execute() method
    cursor.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    #Closing the connection
    conn.close()
    return df

In [20]:
submitted_to_publishers_code = 7
editing_code = 3
writing_code = 12
under_review_code = 13
ready_for_review_code = 4

submitted_to_publishers_df = get_column(submitted_to_publishers_code,'submitted_to_publishers',str_start_date,str_end_date)

editing_df = get_column(editing_code,'editing',str_start_date,str_end_date)
editing_df.drop('publisher',axis=1,inplace=True)

writing_df = get_column(writing_code,'writing',str_start_date,str_end_date)
writing_df.drop('publisher',axis=1,inplace=True)

under_review_df = get_column(under_review_code,'under_review',str_start_date,str_end_date)
under_review_df.drop('publisher',axis=1,inplace=True)

ready_for_review_df = get_column(ready_for_review_code,'ready_for_review',str_start_date,str_end_date)
ready_for_review_df.drop('publisher',axis=1,inplace=True)

### Function for available to write

In [21]:
def get_available_to_write_column(column_name,str_start_date,str_end_date):
    #establishing the connection
    conn = psycopg2.connect(
       database="ee-dashboard", user='mustafa', password='qNNsfVEX!^AA', host='35.245.120.183', port= '5432'
    )

    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()

    # Write query
    query = f"""
    select publishers.name as publisher, COUNT(actions_content.articleid) as {column_name}
    from actions_content
    inner join articles on actions_content.articleid = articles.id and articles.contentsource = 2
    and (actions_content.actionresult = 1 or actions_content.actionresult = 5)
    and actions_content.createdat between '{str_start_date}' and '{str_end_date}'  
    right join publishers on publishers.id = articles.publisherid
    group by publishers.name
    order by publishers.name
    """

    #Executing an MYSQL function using the execute() method
    cursor.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    #Closing the connection
    conn.close()
    return df

In [22]:
available_to_write_df = get_available_to_write_column('available_to_write',str_start_date,str_end_date)
available_to_write_df.drop('publisher',axis=1,inplace=True)

### Function for available to edit

In [23]:
### Function for available to edit

def get_available_to_edit_column(column_name,str_start_date,str_end_date):
    #establishing the connection
    conn = psycopg2.connect(
       database="ee-dashboard", user='mustafa', password='qNNsfVEX!^AA', host='35.245.120.183', port= '5432'
    )

    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()

    # Write query
    query = f"""
    select publishers.name as publisher, COUNT(actions_content.articleid) as {column_name}
    from actions_content
    inner join articles on actions_content.articleid = articles.id and articles.contentsource != 2
    and (actions_content.actionresult = 2 or actions_content.actionresult = 5)
    and actions_content.createdat between '{str_start_date}' and '{str_end_date}'  
    right join publishers on publishers.id = articles.publisherid
    group by publishers.name
    order by publishers.name
    """

    #Executing an MYSQL function using the execute() method
    cursor.execute(query)
    df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    #Closing the connection
    conn.close()
    return df

In [24]:
available_to_edit_df = get_available_to_edit_column('available_to_edit',str_start_date,str_end_date)
available_to_edit_df.drop('publisher',axis=1,inplace=True)

### Combine tables

In [25]:
final_df = pd.concat([submitted_to_publishers_df,editing_df,writing_df,under_review_df,
                      ready_for_review_df,available_to_write_df,available_to_edit_df], axis=1)

In [26]:
final_df

,publisher,submitted_to_publishers,editing,writing,under_review,ready_for_review,available_to_write,available_to_edit
0,21 Oak,0,32,31,72,74,8,24
1,21 Oak CA,0,1,14,1,1,0,14
2,220 TriathlonUK,0,0,0,0,0,0,0
3,22 Words,0,84,69,84,84,0,69
4,Advnture,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
502,Yahoo Life,12,0,11,12,12,11,0
503,Yoga Journal,0,2,0,3,2,1,0
504,Young Choppers,0,0,0,0,0,0,0
505,YourHomeStyleUK,0,0,0,0,0,0,0


In [28]:
final_df.to_csv('7dayarticlestatus.csv',index=False)